In [25]:
# Computations and plots
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter # Method 2 below

# MCMC
import corner
import emcee

%matplotlib inline
# Resolution of spectrometer: 0.5 nm

# Importing dataset

The moon data is from 28/03/2023, while the sky data is from 04/04/2023. The data was collected on top of Rutherford building in Montreal with a celestron telescope and spectromer attached with a 3D custom piece. 

In [26]:
os.chdir("/Users/antoineparadis/Desktop/School/Semester4/PHYS321/PHYS321_CodingLabs2024/FinalProject")
moon_data = pd.read_csv("moon_spectra.csv")
sky_data = pd.read_csv("sky_spectra.csv")

# Normalizing dips for MCMC fitting (Moon data)

The general idea is to obtain a smooth, fitted function to the data and then subtract it from the raw spectra in order to isolate the dips. Normal MCMC can not be applied on the absorption lines directly because of the asymmetric profile. Also, because of atmospheric loss in the spectra, it is not wise to try and fit a planck function.

## Obtain a smoothed curve and the substracting this from the data (using the Savitzky-Golay filter)

In [27]:
# Apply Savitzky-Golay filter, 167 window size (10 windows in 1680 total data points), second order polynomial fit 
smoothed_y = savgol_filter(moon_data['average'], window_length=167, polyorder=2) 
delta2 = moon_data['average']-smoothed_y

# Balmer $\alpha$ (H, n=3 -> n=2)

We are now ready to run MCMC with an inverse lorentztian to fit. First, we define the necessary functions

In [28]:
def inv_lorentz(x,params):
    '''Inverse lorentzian function with offset parameter
    Input:
        x (float np.array): independant variables
        params (float np.array): 
            a -> midheigth width of distribution
            b -> median/mode of distribution
            c -> y-axis offset
    Output:
         (float np.array) : value of distribution at xs'''
    a,b,c = params
    fst = 2/(np.pi*a)
    snd = 1 + ((x-b)/(a/2))**2
    return c-(fst/snd)
  

def log_likelihood(params,x,y,yerr):
    '''
    Models the absorption dips with a lorentzian distribution,
    assuming normally distributed errors
    Input:
        params (float np.array): model parameters
        x (float np.array): wavelength data
        y (float np.array): mean-normalized intensity
        yerr (float np.array): mean-normalized intensity error
    Returns:
        like (float np.array): log likelihood of data given params
    '''
    
    model=inv_lorentz(x,params)
    sigma2 = yerr**2
    like = -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))
    return like


def log_prior(params):
    '''
    Prior knowledge of wavelength bounds and midheigth width
    Input:
        params (float np.array): model parameters
    Returns: 
        (float): 0 if params in bounds
        (float): -inf if params out of bounds
    '''
    a,b,c = params
    if 0 < a < 5 and 650. < b < 660. and -1 < c < 1: # from looking at data
        return 0.0 
    return -np.inf # log(0) = -inf

def log_posterior(params, x, y, yerr):
    '''
    Combines log_likelihood and log_prior to give log_posterior probability of params given data 
    Input:
        params (float np.array): model paramaters
        x (float np.array): wavelength data
        y (float np.array): mean-normalized intensity 
        yerr (float np.array): mean-normalized intensity error 
    Returns:
        (float) : -inf if a,b out of bounds
        (float np.array) : else log_posterior of params given data
    '''
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, x, y, yerr)

Next, we define initial conditions, create walkers, and run the MCMC algorithm

In [30]:
# Initial conditions (this cell takes long too run)

# Data (choose delta2, i.e smoothing out method)
y_balmer = delta2[625:642]
x_balmer = moon_data['wavelength'][625:642]
err_balmer = moon_data['error'][625:642]

ndi = 3  # number of parameters in the model
nw = 32  # number of MCMC walkers
nst = 5000  # number of MCMC steps to take

# Set initial params guesses and add some small fluctuation for all walkers 
para_guesses = np.zeros((nw, ndi))
para_guesses[:, 0] =  2.5 # mid-heigth width (a)
para_guesses[:, 1] =  656 # initial wavelength value (b)
para_guesses[:, 2] =  0.05 # offset (c)
para_guesses += 0.01 * np.random.rand(nw, ndi) # random variations for the 32 walkers to start off

# Run MCMC and get flattened chain
balmer_moon_sample = emcee.EnsembleSampler(nw, ndi, log_posterior, args=[x_balmer,y_balmer,err_balmer])
balmer_moon_sample.run_mcmc(para_guesses, nst, progress = True)
samples = balmer_moon_sample.get_chain()

100%|███████████████████████████████████████| 5000/5000 [03:40<00:00, 22.63it/s]


In [32]:
# Retrieve fit value of balmer absorption line
balmer_w = np.mean(samples[:, :,1])
balmer_w_err = np.std(samples[:, :,1])

print("Note that parameter sets are relatively constrained, as seen in corner plot")
print("Wavelength α balmer fit: ", round(balmer_w,4),"+-",round(balmer_w_err,3))

Note that parameter sets are relatively constrained, as seen in corner plot
Wavelength α balmer fit:  656.4004 +- 0.096
